# Basic Introduction to TensorFlow

In [23]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
import sys
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [24]:
#Tensors
3 # a rank 0 tensor; this is a scalar with shape []
[1. ,2., 3.] # a rank 1 tensor; this is a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]


3

[1.0, 2.0, 3.0]

[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]

[[[1.0, 2.0, 3.0]], [[7.0, 8.0, 9.0]]]

# Constants

In [25]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const_2:0", shape=(), dtype=float32) Tensor("Const_3:0", shape=(), dtype=float32)


In [26]:
#Notice that printing the nodes does not output the values 3.0 and 4.0 as you might expect. 
#Instead, they are nodes that, when evaluated, would produce 3.0 and 4.0, respectively. 
#To actually evaluate the nodes, we must run the computational graph within a session. 

sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [27]:
#more complicated computations
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ",sess.run(node3))

node3:  Tensor("Add_4:0", shape=(), dtype=float32)
sess.run(node3):  7.0


# Placeholders

In [28]:
#A graph can be parameterized to accept external inputs, known as placeholders.
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

In [29]:
print(sess.run(adder_node, {a: 3, b:4.5}))
print(sess.run(adder_node, {a: [1,3], b: [2, 4]}))

7.5
[3. 7.]


In [30]:
#more complex computations
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b:4.5}))

22.5


# Variables

In [31]:
#In ML we typically want a model that can take arbitrary inputs.
#To make the model trainable, we need to be able to modify the graph to get new outputs with the same input. 
#Variables allow us to add trainable parameters to a graph. 

In [32]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

In [33]:
#Constants are initialized when you call tf.constant, and their value can never change. 
#By contrast, variables are not initialized when you call tf.Variable. To initialize 
#all the variables in a TensorFlow program, you must explicitly call a special 
#operation as follows

In [34]:
init = tf.global_variables_initializer()
sess.run(init)

In [35]:
#Since x is a placeholder, we can evaluate linear_model 
#for several values of x simultaneously as follows
print(sess.run(linear_model, {x:[1,2,3,4]}))

[0.         0.3        0.6        0.90000004]


# How accurate is the model?

In [36]:
#We created a model. How good it is?
#To evaluate the model on training data, we need a y placeholder to provide the desired values, 
#and we need to write a loss function.

In [37]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
#tf.reduce_sum sums all the squared errors to create a single scalar 
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))
print(sess.run(b))
print(sess.run(W))

23.66
[-0.3]
[0.3]


In [38]:
#let us improve the model manually
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))
print(sess.run(b))
print(sess.run(W))

[array([-1.], dtype=float32), array([1.], dtype=float32)]

0.0
[1.]
[-1.]


In [39]:
#Yay! we rightly guessed the values of w and b

# Learning our first TensorFlow model

In [40]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
  sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})

print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([0.9999908], dtype=float32)]


# Complete Program -- Linear Regression Model

In [1]:
import numpy as np
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)
# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
# training data
x_train = [1,2,3,4]
y_train = [0,-1,-2,-3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) 
for i in range(1000):
  sess.run(train, {x:x_train, y:y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x:x_train, y:y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [0.9999908] loss: 5.6999738e-11
